Before we let these guys, let's test a couple epochs just to get a baseline ...

these are on 1 CPU machines ...

In [21]:
import utils
import models
import numpy as np
from time import time
from datetime import datetime
from keras.callbacks import EarlyStopping, CSVLogger # maybe more
import json

def run_CNN_model():

    BATCH_SIZE = 25
    EPOCHS = 2

    # choose hyperparameters
    d1 = np.random.randint(0,3)
    d2 = np.random.randint(0,3)
    filters = np.random.randint(50,1001)
    kernel_size = np.random.randint(1,11)
    hidden_units = np.random.randint(25,251)
    dropout_rate = round(np.random.uniform(0,1,size=None), 3)
    embed_vec = np.random.choice(['glove100', 'glove200', 'glove300', 'numberbatch', 'lexvec'])

    ## start the timer
    start = time()

    # load the corpus
    corpus = utils.AMI_Corpus(seed = 75, embed_vec = embed_vec)

    cnn = models.CNN(corpus = corpus, d1 = d1, d2 = d2,
                       batch_size = BATCH_SIZE,
                       filters = filters,
                       kernel_size = kernel_size,
                       hidden_units = hidden_units,
                       dropout_rate = dropout_rate)

    cnn.model.compile(optimizer = 'adagrad', metrics = ['acc'], loss = 'categorical_crossentropy')

    # create our generators
    ug_train = utils.UtteranceGenerator(corpus, "train", batch_size = BATCH_SIZE, sequence_length = (d1 + d2 + 1))
    ug_val = utils.UtteranceGenerator(corpus, "val", batch_size = BATCH_SIZE, sequence_length = (d1 + d2 + 1))

    # create keras callbacks
    es = EarlyStopping(monitor='val_loss', patience=1, verbose=0)

    right_now = datetime.now().isoformat() # timestamp
    csv_logger = CSVLogger('logs/TEST_cnn_history_' + right_now) # log epochs in case I want to look back later

    try:
    # note to self, maybe change validation_steps and validation_freq
        history = cnn.model.fit_generator(ug_train, epochs=EPOCHS, verbose=1, callbacks=[es, csv_logger],
                                          validation_data=ug_val, validation_steps=100, #validation_freq=1,
                                          use_multiprocessing=False, shuffle=True)
    except:
        print(d1, d2, filters, kernel_size, hidden_units, dropout_rate, embed_vec)
        raise Exception()

    results = {"d1":d1, "d2":d2, "filters":filters, "kernel_size":kernel_size, "hidden_units":hidden_units, "dropout_rate":dropout_rate,
               "embed_vec":embed_vec, "acc":history.history['acc'][-5:], "val_acc":history.history['acc'][-1:],
               "loss":history.history['loss'][-1:], "val_loss":history.history['val_loss'][-1:], time:time() - start}

    with open("logs/TEST_cnn_params_" + right_now + ".json", "w") as f:
        json.dump(results, f)

In [22]:
from importlib import reload
reload(utils)
reload(models)
run_CNN_model()

Loading corpus from /home/jake_miller/final/nlp-final-project/data
Begin corpus post-processing ...
Splitting corpus into training and test ...
Creating vocabulary from training set ...
Found 8379 unique words.
Building initial embedding matrix ...
(8381, 300)
loading pretrained vectors from glove.6B.300d.txt
Epoch 1/2
 256/3239 [=>............................] - ETA: 36:07 - loss: 11.8368 - acc: 0.26342 0 986 8 168 0.239 glove300


Exception: 

on one CPU, with a batch size of 25, we were looking at around 37 (!) minutes for one f'cking epoch (regardless, it seems to be running, so I'm killing it. We'll run them for real with a bigger machine.)

### let's test the LSTM

In [27]:
def run_lstm_model():

    BATCH_SIZE = 100
    EPOCHS = 50

    # choose hyperparameters
    num_layers = np.random.randint(2,16) # lstm layers
    hidden_state_size = np.random.randint(100,400)
    dropout_rate = round(np.random.uniform(0,1,size=None), 3)
    embed_vec = np.random.choice(['glove100', 'glove200', 'glove300', 'numberbatch', 'lexvec'])

    ## start the timer
    start = time()

    # load the corpus
    corpus = utils.AMI_Corpus(seed = 75, embed_vec = embed_vec)

    lstm = models.LSTMSoftmax(corpus,
                              batch_size = BATCH_SIZE,
                              num_layers = num_layers,
                              dropout_rate = dropout_rate,
                              hidden_state_size = hidden_state_size,
                              stateful = False, bidirectional = False, trainable_embed = False)

    lstm.model.compile(optimizer = 'adagrad', metrics = ['acc'], loss = 'categorical_crossentropy')

    # create our generators
    ug_train = utils.UtteranceGenerator(corpus, "train", batch_size = BATCH_SIZE)
    ug_val = utils.UtteranceGenerator(corpus, "val", batch_size = BATCH_SIZE)

    # create keras callbacks
    es = EarlyStopping(monitor='val_loss', patience=5, verbose=0)

    right_now = datetime.now().isoformat() # timestamp
    csv_logger = CSVLogger('logs/lstm_history_' + right_now) # log epochs in case I want to look back later

    # note to self, maybe change validation_steps and validation_freq
    history = lstm.model.fit_generator(ug_train, epochs=EPOCHS, verbose=1, callbacks=[es, csv_logger],
                      validation_data=ug_val, validation_steps=100, #validation_freq=1,
                      use_multiprocessing=False, shuffle=True)

    results = {"num_layers":num_layers, "hidden_state_size":hidden_state_size,"dropout_rate":dropout_rate,
               "embed_vec":embed_vec, "acc":history.history['acc'][-5:], "val_acc":history.history['acc'][-5:],
               "loss":history.history['loss'][-5:], "val_loss":history.history['val_loss'][-5:], time:time() - start}

    with open("logs/lstm_params_" + right_now + ".json", "w") as f:
        json.dump(results, f)

In [28]:
reload(utils)
reload(models)
run_lstm_model()

Loading corpus from /home/jake_miller/final/nlp-final-project/data
Begin corpus post-processing ...
Splitting corpus into training and test ...
Creating vocabulary from training set ...
Found 8379 unique words.
Building initial embedding matrix ...
(8381, 200)
loading pretrained vectors from glove.6B.200d.txt
Epoch 1/50
 81/810 [==>...........................] - ETA: 54:45 - loss: 2.2053 - acc: 0.2490

KeyboardInterrupt: 

We're gonna need a bigger boat! This one estimates at roughly an hour per epoch